### Загрузим и обработаем датасет

In [1]:
from  load_data import Load_Process_Dataset

D = Load_Process_Dataset('adult')
D.start_load()

Start processing dataset adult from UCI.
Aready downloaded.
adult (32561, 15) (16281, 15) (32561, 15)
Numerical (32561, 6)
Categorical (32561, 8)
Processing and Saving adult Successfully!
adult
Total 48842
Train 32561
Test 16281
Num 6
Cat 9


### train VAE


#### params

In [1]:
LR = 1e-3
WD = 0
D_TOKEN = 4
TOKEN_BIAS = True

In [2]:
N_HEAD = 1
FACTOR = 32
NUM_LAYERS = 2

In [3]:
import torch
if torch.cuda.is_available():
    my_device = torch.device('cuda')
else:
    my_device = torch.device('cpu')
print('Device: {}'.format(my_device))

Device: cuda


In [4]:
from model_train.train_vae import VAE_main

vae_train = VAE_main(name = 'adult', LR = 1e-3, WD = 0, D_TOKEN = 4, N_HEAD = 1, FACTOR = 32, NUM_LAYERS = 2)

In [6]:
vae_train.tabsyn_vae_main(device = my_device, num_epochs = 5)

No NaNs in numerical features, skipping


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


self.category_embeddings.weight.shape=torch.Size([104, 4])
self.category_embeddings.weight.shape=torch.Size([104, 4])


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch: 0, beta = 0.010000, Train MSE: 9.334653, Train CE:2.433241, Train KL:0.467871, Val MSE:7.292939, Val CE:2.475718, Train ACC:0.189509, Val ACC:0.188679
Training time: 0.1186 mins
Successfully load and save the model!
Successfully save pretrained embeddings in disk!


###train TabSyn

In [7]:
from model_train.train_tabsyn import tabsyn_main
tabsyn_main(my_device, 'adult', 5)

model
MLPDiffusion(
  (proj): Linear(in_features=60, out_features=1024, bias=True)
  (mlp): Sequential(
    (0): Linear(in_features=1024, out_features=2048, bias=True)
    (1): SiLU()
    (2): Linear(in_features=2048, out_features=2048, bias=True)
    (3): SiLU()
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): SiLU()
    (6): Linear(in_features=1024, out_features=60, bias=True)
  )
  (map_noise): PositionalEmbedding()
  (time_embed): Sequential(
    (0): Linear(in_features=1024, out_features=1024, bias=True)
    (1): SiLU()
    (2): Linear(in_features=1024, out_features=1024, bias=True)
  )
)
the number of parameters 10616892
Epoch:0, Loss:0.640062503438189
Time:  4.08256721496582


###Генерируем синтетику

In [8]:
from generate_synthetic import sample_main
sample_main()

No NaNs in numerical features, skipping
(32561, 9)
Time: 14.878840923309326
Saving sampled data to synthetic/adult/synthetic.csv


### Сравнение на классификаторе

In [9]:
pip install prdc

In [5]:
from metrics.mle import MLE

Синтетика

In [6]:
dataname = 'adult'

train_path = f'synthetic/{dataname}/synthetic.csv'
test_path = f'synthetic/{dataname}/test.csv'
MLE(train_path, test_path, dataname)

  0%|          | 0/36 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:43:53] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:43:53] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classificatio

Saving scores to  eval/mle/adult/model.json


Трушные

In [8]:
dataname = 'adult'
model = 'real'
train_path = f'synthetic/{dataname}/{model}.csv'
test_path = f'synthetic/{dataname}/test.csv'

MLE(train_path, test_path, dataname)

  0%|          | 0/36 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:44:28] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
  3%|▎         | 1/36 [00:00<00:17,  1.97it/s]/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:44:28] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [20:44:29] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA i

Saving scores to  eval/mle/adult/model.json


In [13]:
import json
import pandas as pd
with open('/content/eval/mle/adult/synt.json', 'r') as myFile:
    synthetic = json.load(myFile)

In [14]:
with open('/content/eval/mle/adult/real.json', 'r') as myFile:
    real = json.load(myFile)

In [15]:
df_real = pd.DataFrame({key: value["XGBClassifier"] for key, value in real.items()})
df_synt = pd.DataFrame({key: value["XGBClassifier"] for key, value in synthetic.items()})

df_real = df_real.rename(index={key: f'{key}_real' for key in df_real.index})
df_synt = df_synt.rename(index={key: f'{key}_synt' for key in df_synt.index})

df = pd.concat([df_real, df_synt])
df.sort_index(inplace=True)

df

,best_f1_scores,best_weighted_scores,best_auroc_scores,best_acc_scores,best_avg_scores
accuracy_real,0.873472,0.873472,0.874946,0.873472,0.873472
accuracy_synt,0.750200,0.750200,0.747190,0.795713,0.724955
binary_f1_real,0.708133,0.708133,0.712754,0.708133,0.708133
binary_f1_synt,0.397214,0.397214,0.464759,0.517971,0.373531
roc_auc_real,0.927122,0.927122,0.927578,0.927122,0.927122
roc_auc_synt,0.760199,0.760199,0.772563,0.808089,0.714721
weighted_f1_real,0.757999,0.757999,0.761729,0.757999,0.757999
weighted_f1_synt,0.502392,0.502392,0.552105,0.601222,0.479896


### Alpha Precision and Beta Recall

In [ ]:
pip install synthcity

In [1]:
from metrics.alpha_beta import Alpha_Beta
Alpha_Beta()

=========== All Features ===========
Data shape:  (32561, 110)
alpha precision: 0.067103, beta recall: 0.000111


### Density estimation of single column and pair-wise correlation

In [ ]:
pip install sdmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.4 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
import os

import json

# Metrics
from sdmetrics.reports.single_table import QualityReport, DiagnosticReport


def reorder(real_data, syn_data, info):
    num_col_idx = info['num_col_idx']
    cat_col_idx = info['cat_col_idx']
    target_col_idx = info['target_col_idx']

    task_type = info['task_type']
    if task_type == 'regression':
        num_col_idx += target_col_idx
    else:
        cat_col_idx += target_col_idx

    real_num_data = real_data[num_col_idx]
    real_cat_data = real_data[cat_col_idx]

    new_real_data = pd.concat([real_num_data, real_cat_data], axis=1)
    new_real_data.columns = range(len(new_real_data.columns))

    syn_num_data = syn_data[num_col_idx]
    syn_cat_data = syn_data[cat_col_idx]

    new_syn_data = pd.concat([syn_num_data, syn_cat_data], axis=1)
    new_syn_data.columns = range(len(new_syn_data.columns))


    metadata = info['metadata']

    columns = metadata['columns']
    metadata['columns'] = {}

    inverse_idx_mapping = info['inverse_idx_mapping']


    for i in range(len(new_real_data.columns)):
        if i < len(num_col_idx):
            metadata['columns'][i] = columns[num_col_idx[i]]
        else:
            metadata['columns'][i] = columns[cat_col_idx[i-len(num_col_idx)]]


    return new_real_data, new_syn_data, metadata


In [4]:
dataname = 'adult'
model = 'model'

syn_path = 'synthetic/adult/synthetic.csv'

real_path = f'synthetic/{dataname}/real.csv'

data_dir = f'data/{dataname}'
print(syn_path)

with open(f'{data_dir}/info.json', 'r') as f:
    info = json.load(f)

syn_data = pd.read_csv(syn_path)
real_data = pd.read_csv(real_path)

save_dir = f'eval/density/{dataname}/{model}'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

real_data.columns = range(len(real_data.columns))
syn_data.columns = range(len(syn_data.columns))

metadata = info['metadata']
metadata['columns'] = {int(key): value for key, value in metadata['columns'].items()}

new_real_data, new_syn_data, metadata = reorder(real_data, syn_data, info)

qual_report = QualityReport()
qual_report.generate(new_real_data, new_syn_data, metadata)

diag_report = DiagnosticReport()
diag_report.generate(new_real_data, new_syn_data, metadata)

quality =  qual_report.get_properties()
diag = diag_report.get_properties()

Shape = quality['Score'][0]
Trend = quality['Score'][1]

with open(f'{save_dir}/quality.txt', 'w') as f:
    f.write(f'{Shape}\n')
    f.write(f'{Trend}\n')

Quality = (Shape + Trend) / 2

shapes = qual_report.get_details(property_name='Column Shapes')
trends = qual_report.get_details(property_name='Column Pair Trends')
# coverages = diag_report.get_details('Coverage')


shapes.to_csv(f'{save_dir}/shape.csv')
trends.to_csv(f'{save_dir}/trend.csv')
# coverages.to_csv(f'{save_dir}/coverage.csv')

synthetic/adult/synthetic.csv
Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 15/15 [00:00<00:00, 30.26it/s]|
Column Shapes Score: 51.39%

(2/2) Evaluating Column Pair Trends: |██████████| 105/105 [00:03<00:00, 33.77it/s]|
Column Pair Trends Score: 15.97%

Overall Score (Average): 33.68%

Generating report ...

(1/2) Evaluating Data Validity: |██████████| 15/15 [00:00<00:00, 113.89it/s]|
Data Validity Score: 100.0%

(2/2) Evaluating Data Structure: |██████████| 1/1 [00:00<00:00, 146.52it/s]|
Data Structure Score: 100.0%

Overall Score (Average): 100.0%



In [5]:
shapes = qual_report.get_details(property_name='Column Shapes')
trends = qual_report.get_details(property_name='Column Pair Trends')
coverages_srt = diag_report.get_details('Data Structure')
coverages_val = diag_report.get_details('Data Validity')

In [ ]:
pip install plotly==5.3.1

In [ ]:
fig = qual_report.get_visualization(property_name='Column Shapes')
fig.show(renderer = 'svg')

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [ ]:
fig = qual_report.get_visualization(property_name='Column Pair Trends')
fig.show(renderer = 'svg')

In [ ]:
fig = diag_report.get_visualization(property_name = 'Data Validity')
fig.show(renderer = 'svg')

In [ ]:
from sdmetrics.visualization import get_column_plot
for i in range(0, 6):
    fig = get_column_plot(
        real_data=new_real_data,
        synthetic_data=new_syn_data,
        column_name= i,
        plot_type='distplot'
    )

    fig.show(renderer = 'svg')

### Pricavy protection: Distance to Closest Record (DCR)

In [1]:
from metrics.dcr import DCR
DCR()

DCR Score, a value closer to 0.5 is better
adult-model, DCR Score = 0.6164429839378397
